In [2]:
import pandas as pd

import sys
import ujson
import pprint
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# For displaying google maps
import gmplot
from IPython.display import IFrame

# Second (better?) google map, only inline so far
import gmaps

# For summary stats
import scipy
from scipy import stats

from influxdb import DataFrameClient

pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

In [4]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')


time: 33.4 ms


In [5]:
# Read config from local file
GMAP_API_KEY = None

sys.path.append('/Users/patrickmulrooney/Desktop/')
import capstone_config

time: 9.28 ms


In [7]:
GMAP_API_KEY = capstone_config._GMAP_API_KEY

time: 1.05 ms


In [9]:
dbname = 'endomondo'
client = DataFrameClient('localhost', 8086, 'root', 'root', dbname)

time: 2.31 ms


In [ ]:
endoHRrun = client.query("select * from run")